## Задача 5

### Решить дифференциальное уравнение методом сеток (задание - 5.5, стр.46, варианты - 5.6, стр.48, по второй части методички).

### Вариант 7

Входные данные

In [1]:
import numpy as np
from pandas import DataFrame

def alpha_1(t):
    return 0

def alpha_2(t):
    return -1

def beta_1(t):
    return 1

def beta_2(t):
    return 0

def alpha(t):
    return 0

def beta(t):
    return u(1, t)

def phi(x: float):
    return u(x, 0)

def p(x: float):
    return x + 1

def b(x, t):
    return 0

def c(x, t):
    return 0

def u(x, t):
    return (x ** 3) + (t ** 3)

def f(x, t):
    return 3 * (t ** 2) - 9 * (x ** 2) - 6 * x

A = 2
M = [5, 10, 20, 40, 80, 160]

Алгоритм явной схемы

In [2]:
def difference_operator(h: float, tau: float, i: int, k: int, table):
    ans = p((i + 0.5) * h) * (table[k, i + 1] - table[k, i]) / (h ** 2)
    ans -= p((i - 0.5) * h) * (table[k, i] - table[k, i - 1]) / (h ** 2)
    ans += b(i * h, k * tau) * (table[k, i + 1] - table[k, i - 1]) / (2 * h)
    ans += c(i * h, k * tau) * table[k, i]
    return ans


def explicit_schema(h: float, tau: float, m: int, n: int):
    ans_table = np.zeros((m + 1, n + 1))
    for i in range(n + 1):
        ans_table[0, i] = phi(i * h)

    for k in range(1, m + 1):
        current_t = k * tau
        for i in range(1, n):
            ans_table[k, i] = ans_table[k-1, i]

            ans_table[k, i] += tau * (difference_operator(h, tau, i, k - 1, ans_table) + f(h * i, (k - 1) * tau))

        ans_table[k, 0] = (alpha(current_t) + alpha_2(current_t) * (4 * ans_table[k, 1] - ans_table[k, 2]) / (2 * h)) / \
                          (alpha_1(current_t) + alpha_2(current_t) * 3 / (2 * h))
        ans_table[k, n] = (beta(current_t) + beta_2(current_t) * (4 * ans_table[k, n - 1] - ans_table[k, n - 2]) / (2 * h)) / \
                          (beta_1(current_t) + beta_2(current_t) * 3 / (2 * h))
    return ans_table

Алгоритм неявной схемы

In [3]:
def implicit_schema(h: float, tau: float, m: int, n: int, sigma: float):
    ans_table = np.zeros((m + 1, n + 1))
    for i in range(n + 1):
        ans_table[0, i] = phi(i * h)

    for k in range(1, m + 1):
        current_t = k * tau

        A = np.zeros(n + 1)
        B = np.zeros(n + 1)
        C = np.zeros(n + 1)
        G = np.zeros(n + 1)

        A[0] = 0.0
        B[0] = -alpha_1(current_t) - alpha_2(current_t) / h
        C[0] = -alpha_2(current_t) / h
        G[0] = alpha(current_t)

        A[n] = -beta_2(current_t) / h
        B[n] = -beta_1(current_t) - beta_2(current_t) / h
        C[n] = 0.0
        G[n] = beta(current_t)

        for i in range(1, n):
            current_x = i * h
            A[i] = sigma * (p(current_x - 0.5 * h) / (h ** 2) - b(current_x, current_t) / (2 * h))
            B[i] = sigma * (p(current_x + 0.5 * h) / (h ** 2) + p(current_x - 0.5 * h) / (h ** 2) - c(current_x, current_t)) + 1 / tau
            C[i] = sigma * (p(current_x + 0.5 * h) / (h ** 2) + b(current_x, current_t) / (2 * h))
            G[i] = -1 / tau * ans_table[k - 1, i] - (1 - sigma) * difference_operator(h, tau, i, k - 1, ans_table) - \
                   f(current_x, current_t - (1 - sigma) * tau)

        s = np.zeros(n + 1)
        t = np.zeros(n + 1)

        s[0] = C[0] / B[0]
        t[0] = -G[0] / B[0]

        for i in range(1, n + 1):
            s[i] = C[i] / (B[i] - A[i] * s[i - 1])
            t[i] = (A[i] * t[i - 1] - G[i]) / (B[i] - A[i] * s[i - 1])

        ans_table[k, n] = t[n]
        for i in reversed(range(0, n)):
            ans_table[k, i] = s[i] * ans_table[k, i + 1] + t[i]

    return ans_table

Крупная сетка для явной схемы

In [4]:
n = 5
h = 1 / n
m = 10
tau = 0.1 / m

ans_table = explicit_schema(h, tau, m, n)
columns = [f"{k * h}" for k in range(n + 1)]
rows = [f"{k * tau}" for k in range(m + 1)]
table = DataFrame(ans_table, columns=columns, index=rows)
table

,0.0,0.2,0.4,0.6000000000000001,0.8,1.0
0.0,0.000000,0.008000,0.064000,0.216000,0.512000,1.000000
0.01,-0.010267,0.008400,0.064400,0.216400,0.512400,1.000001
0.02,-0.013775,0.005870,0.064803,0.216803,0.512613,1.000008
0.03,-0.016044,0.004033,0.064262,0.217134,0.512818,1.000027
0.04,-0.017863,0.002513,0.063642,0.217100,0.513004,1.000064
0.05,-0.019357,0.001224,0.062970,0.216960,0.513046,1.000125
0.06,-0.020632,0.000107,0.062324,0.216725,0.513047,1.000216
0.07,-0.021728,-0.000868,0.061712,0.216470,0.513024,1.000343
0.08,-0.022677,-0.001719,0.061155,0.216218,0.513012,1.000512
0.09,-0.023496,-0.002457,0.060661,0.215999,0.513029,1.000729


Крупная сетка для неявной схемы

In [5]:
n = 5
h = 1 / n
m = 10
tau = 0.1 / m
sigma = 0.5

ans_table = implicit_schema(h, tau, m, n, sigma)
columns = [f"{k * h}" for k in range(n + 1)]
rows = [f"{k * tau}" for k in range(m + 1)]
table = DataFrame(ans_table, columns=columns, index=rows)
table

,0.0,0.2,0.4,0.6000000000000001,0.8,1.0
0.0,0.000000,0.008000,0.064000,0.216000,0.512000,1.000000
0.01,0.009363,0.009363,0.064517,0.216407,0.512336,1.000001
0.02,0.011480,0.011480,0.065308,0.216850,0.512594,1.000008
0.03,0.013264,0.013264,0.066300,0.217365,0.512844,1.000027
0.04,0.014850,0.014850,0.067350,0.217952,0.513123,1.000064
0.05,0.016309,0.016309,0.068410,0.218594,0.513441,1.000125
0.06,0.017681,0.017681,0.069468,0.219277,0.513801,1.000216
0.07,0.018996,0.018996,0.070522,0.219995,0.514204,1.000343
0.08,0.020274,0.020274,0.071576,0.220743,0.514649,1.000512
0.09,0.021532,0.021532,0.072637,0.221523,0.515138,1.000729


Таблица для явной схемы

In [6]:
def choose_tau(h: float):
    for m in M:
        tau = 0.1 / m
        if A * tau / (h ** 2) <= 0.5:
            return tau

def make_real_table(h: float, tau: float, m: int, n: int):
    table = np.zeros((m + 1, n + 1))
    for k in range(m + 1):
        for i in range(n + 1):
            table[k, i] = u(i * h, k * tau)
    return table


rows = []
for h in [0.2, 0.1, 0.05]:
    tau = choose_tau(h)
    m = int(0.1 / tau)
    n = int(1 / h)
    real_table = make_real_table(h, tau, m, n)
    explicit_table = explicit_schema(h, tau, m, n)
    delta = np.abs(real_table - explicit_table).max()
    if h == 0.2:
        rows.append([h, tau, delta, "---"])
    else:
        two_n = int(1 / (h * 2))
        two_explicit_table = explicit_schema(h * 2, tau, m, two_n)
        minimized_table = np.array([[explicit_table[k, i] for i in range(0, n + 1, 2)] for k in range(m + 1)])
        two_delta = np.abs(minimized_table - two_explicit_table).max()
        rows.append([h, tau, delta, two_delta])

table = DataFrame(rows, columns=["h", "tau", "||J - u||", "||u(2h,tau) - u(h, tau)"])
table

,h,tau,||J - u||,"||u(2h,tau) - u(h, tau)"
0,0.20,0.010000,0.025195,---
1,0.10,0.002500,0.005929,0.019886
2,0.05,0.000625,0.001451,0.004495


Таблица для неявной схемы с $\sigma = 0.0$

In [7]:
m = 10
rows = []
sigma = 0

for h in [0.2, 0.1, 0.05]:
    tau = 1 / m
    n = int(1 / h)
    real_table = make_real_table(h, tau, m, n)
    implicit_table = implicit_schema(h, tau, m, n, sigma)
    delta = np.abs(real_table - implicit_table).max()
    if h == 0.2:
        rows.append([h, tau, delta, "---"])
    else:
        two_n = int(1 / (h * 2))
        two_implicit_table = implicit_schema(2 * h, tau, m, two_n, sigma)
        minimized_table = np.array([[implicit_table[k, i] for i in range(0, n + 1, 2)] for k in range(m + 1)])
        two_delta = np.abs(two_implicit_table - minimized_table).max()
        rows.append([h, tau, delta, two_delta])
table = DataFrame(rows, columns=["h", "tau", "||J - u||", "||u(2h,tau) - u(h, tau)"])
table

,h,tau,||J - u||,"||u(2h,tau) - u(h, tau)"
0,0.20,0.1,5.633848e+06,---
1,0.10,0.1,6.681333e+10,66818144687.671974
2,0.05,0.1,4.595865e+17,459586436465525056.0


Таблица для неявной схемы с $\sigma = 0.5$

In [8]:
m = 100
rows = []
sigma = 0.5

for h in [0.2, 0.1, 0.05]:
    tau = 1 / m
    n = int(1 / h)
    real_table = make_real_table(h, tau, m, n)
    implicit_table = implicit_schema(h, tau, m, n, sigma)
    delta = np.abs(real_table - implicit_table).max()
    if h == 0.2:
        rows.append([h, tau, delta, "---"])
    else:
        two_n = int(1 / (h * 2))
        two_implicit_table = implicit_schema(2 * h, tau, m, two_n, sigma)
        minimized_table = np.array([[implicit_table[k, i] for i in range(0, n + 1, 2)] for k in range(m + 1)])
        two_delta = np.abs(two_implicit_table - minimized_table).max()
        rows.append([h, tau, delta, two_delta])
table = DataFrame(rows, columns=["h", "tau", "||J - u||", "||u(2h,tau) - u(h, tau)"])
table

,h,tau,||J - u||,"||u(2h,tau) - u(h, tau)"
0,0.20,0.01,0.039817,---
1,0.10,0.01,0.009912,0.029905
2,0.05,0.01,0.002466,0.007446


Таблица для неявной схемы с $\sigma = 1$

In [9]:
m = 100
rows = []
sigma = 1

for h in [0.2, 0.1, 0.05]:
    tau = 1 / m
    n = int(1 / h)
    real_table = make_real_table(h, tau, m, n)
    implicit_table = implicit_schema(h, tau, m, n, sigma)
    delta = np.abs(real_table - implicit_table).max()
    if h == 0.2:
        rows.append([h, tau, delta, "---"])
    else:
        two_n = int(1 / (h * 2))
        two_implicit_table = implicit_schema(2 * h, tau, m, two_n, sigma)
        minimized_table = np.array([[implicit_table[k, i] for i in range(0, n + 1, 2)] for k in range(m + 1)])
        two_delta = np.abs(two_implicit_table - minimized_table).max()
        rows.append([h, tau, delta, two_delta])
table = DataFrame(rows, columns=["h", "tau", "||J - u||", "||u(2h,tau) - u(h, tau)"])
table

,h,tau,||J - u||,"||u(2h,tau) - u(h, tau)"
0,0.20,0.01,0.045518,---
1,0.10,0.01,0.016245,0.02929
2,0.05,0.01,0.009097,0.00716
